# AVI 3 Migration Sample

This is for first steps... of AVI by using Jupyter

https://avinetworks.com/docs/latest/api-guide/overview.html 


https://github.com/avinetworks/sdk/blob/master/python/avi/sdk/README.md

[1.- Initial Login](#1--initial-login)
[2.- Read Operations](#import-libraries)

## Initial Login

In [1]:
from avi.sdk.avi_api import ApiSession
import datetime, time
from requests.packages import urllib3
urllib3.disable_warnings()
import json
import pandas as pd

# Import environment variables with controller information and credentials
from envs.controller_info import session_params as session_env

# Establish a first session with AVI Controller
api = ApiSession(
    controller_ip=session_env['controller_ip'],
    username=session_env['controller_username'],
    password=session_env['controller_password'],
    tenant=session_env['tenant'],
    api_version=session_env['api_version']
    )
# Update headers and api version imported from demo env file with controller version (ensure actual API Version is uses in subsequent requests
session_env['headers']['X-Avi-Version'] = api.remote_api_version['Version']
session_env['api_version'] = api.remote_api_version['Version']

# Create a new session with received AVI API Version
api = ApiSession(
    controller_ip=session_env['controller_ip'],
    username=session_env['controller_username'],
    password=session_env['controller_password'],
    tenant=session_env['tenant'],
    api_version=session_env['api_version']
    )
# Display Session ID to Verify AVI Controller Session Establishment
print('Successful connection to ' + session_env['name'] + '. Session ID:' + api.session_id)

Successful connection to avicontroller. Session ID:ddgs5yrq4lqqzjpfcrdr3tlbz7i8frsx


## 2.- Read Operations

### 2.1 Basic Operations via JSON

Import some required Libraries first

In [4]:
# Extract information from MAIN AND MIGRATED SEGROUP
# MAIN --> Target SE Group where consolditated VS will be placed
# MIGRATED --> SE Group to consolidate into MAIN SE Group after configuration extraction

The first step is to Gather SE Group Information
- Service Engine Group Configuration
- Service Engine Configuration
- Virtual Services Configuration
- Network Services Configuration (if exists)

In [55]:
# Set consolidated name
source_segroup = "MAD-SEG002"

In [167]:
# GET Service Engine Group Configuration

# Define GET request parameters
url_path = "serviceenginegroup"
query = {
   "skip_default": "true",
   "name": source_segroup
}
# Send GET Request
resp = api.get(url_path, params=query)
print ("Request sent to URL: " + resp.url)

# Control Response Status Code
if resp.status_code in range(200, 299):
   print(resp)
   print(resp.reason)
    
   # Convert response JSON into Python Dictionary
   resp_data = json.loads(resp.text)

   # Extract Result
   resp_data = resp_data["results"]
   resp_names = [resp_name["name"] for resp_name in resp_data]
   print()
   print("The following " + url_path + " names has been found:")
   print(resp_names)
else:
    print('Error in GET request '+url_path+' :%s' % resp.text)

# Save Result
migrating_segroup_data = resp_data[0]

Request sent to URL: https://192.168.1.15/api/serviceenginegroup?skip_default=true&name=MAD-SEG002
<Response [200]>
OK

The following serviceenginegroup names has been found:
['MAD-SEG002']


In [169]:
# Extract related virtual services
# GET Virtual Services Related Configuration

seg_data = migrating_segroup_data
# Define GET request parameters
se_group_uuid=seg_data["uuid"]
url_path = "virtualservice"
query = {
   "skip_default": "true",
   "refers_to": f"serviceenginegroup:{se_group_uuid}"
}

# Send GET Request
resp = api.get(url_path, params=query)
print ("Request sent to URL: " + resp.url)

# Control Response Status Code
if resp.status_code in range(200, 299):
   print(resp)
   print(resp.reason)
    
   # Convert response JSON into Python Dictionary
   resp_data = json.loads(resp.text)

   # Extract Result
   resp_data = resp_data["results"]
   resp_names = [resp_name["name"] for resp_name in resp_data]
   print()
   print("The following " + url_path + " names has been found:")
   print(resp_names)
else:
    print('Error in GET request '+url_path+' :%s' % resp.text)

# Save Result
vs_data = resp_data

Request sent to URL: https://192.168.1.15/api/virtualservice?skip_default=true&refers_to=serviceenginegroup%3Aserviceenginegroup-932a4c8c-ee69-4970-94dc-e3d9fc42032f
<Response [200]>
OK

The following virtualservice names has been found:
['vs-example-mad-002-01', 'vs-example-mad-002-02']


In [170]:
# Extract related Service Engine

# GET Service Engine Related Configuration

# Define GET request parameters
se_group_uuid=seg_data["uuid"]
url_path = "serviceengine"
query = {
   "skip_default": "true",
   "refers_to": f"serviceenginegroup:{se_group_uuid}"
}

# Send GET Request
resp = api.get(url_path, params=query)
print ("Request sent to URL: " + resp.url)

# Control Response Status Code
if resp.status_code in range(200, 299):
   print(resp)
   print(resp.reason)
    
   # Convert response JSON into Python Dictionary
   resp_data = json.loads(resp.text)

   # Extract Result
   resp_data = resp_data["results"]
   resp_names = [resp_name["name"] for resp_name in resp_data]
   print()
   print("The following " + url_path + " names has been found:")
   print(resp_names)
else:
    print('Error in GET request '+url_path+' :%s' % resp.text)

# Save Result
migrating_se_data = resp_data

Request sent to URL: https://192.168.1.15/api/serviceengine?skip_default=true&refers_to=serviceenginegroup%3Aserviceenginegroup-932a4c8c-ee69-4970-94dc-e3d9fc42032f
<Response [200]>
OK

The following serviceengine names has been found:
['fd38:9bda:673d:0:20c:29ff:fea7:8358', 'fd38:9bda:673d:0:20c:29ff:fea7:610c']


In [172]:
# Extract Interface Information from first SE
se_data = migrating_se_data

se1_data_vnics = se_data[0]["data_vnics"]
se1_df = pd.DataFrame(se1_data_vnics)
se1_df

,adapter,connected,dhcp_enabled,if_name,ip6_autocfg_enabled,linux_name,mac_address,port_uuid,vrf_ref,vnic_networks
0,Unknown,True,False,eth3,False,eth3,00:0c:29:a7:83:62,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
1,Unknown,True,False,eth4,False,eth6,00:0c:29:a7:83:6c,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
2,Unknown,True,False,eth6,False,eth8,00:0c:29:a7:83:76,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
3,Unknown,True,False,eth8,False,eth1,00:0c:29:a7:83:80,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
4,Unknown,True,False,eth1,False,eth4,00:0c:29:a7:83:8a,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,"[{'ip': {'ip_addr': {'addr': '192.168.1.42', '..."
5,Unknown,True,False,eth2,False,eth7,00:0c:29:a7:83:94,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
6,Unknown,True,False,eth5,False,eth9,00:0c:29:a7:83:9e,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
7,Unknown,True,False,eth7,False,eth2,00:0c:29:a7:83:a8,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
8,Unknown,True,False,eth9,False,eth5,00:0c:29:a7:83:b2,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN


In [173]:
# Extract Interface Information from second SE
se2_data_vnics = se_data[1]["data_vnics"]
se2_df = pd.DataFrame(se2_data_vnics)
se2_df

,adapter,connected,dhcp_enabled,if_name,ip6_autocfg_enabled,linux_name,mac_address,port_uuid,vrf_ref,vnic_networks
0,Unknown,True,False,eth3,False,eth3,00:0c:29:a7:61:16,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
1,Unknown,True,False,eth4,False,eth6,00:0c:29:a7:61:20,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
2,Unknown,True,False,eth6,False,eth8,00:0c:29:a7:61:2a,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
3,Unknown,True,False,eth8,False,eth1,00:0c:29:a7:61:34,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
4,Unknown,True,False,eth1,False,eth4,00:0c:29:a7:61:3e,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,"[{'ip': {'ip_addr': {'addr': '192.168.1.43', '..."
5,Unknown,True,False,eth2,False,eth7,00:0c:29:a7:61:48,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
6,Unknown,True,False,eth5,False,eth9,00:0c:29:a7:61:52,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
7,Unknown,True,False,eth7,False,eth2,00:0c:29:a7:61:5c,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
8,Unknown,True,False,eth9,False,eth5,00:0c:29:a7:61:66,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN


In [174]:
# Select Interface to be migrated (ensure you are picking if_name and not linux_name column)
if_name_to_migrate = "eth1"

se1_data_vnic_to_migrate = [ d for d in se1_data_vnics if d.get("if_name") == if_name_to_migrate]
se1_data_vnic_to_migrate_df = pd.DataFrame(se1_data_vnic_to_migrate)

# Display results in tab format
se1_data_vnic_to_migrate_df

,adapter,connected,dhcp_enabled,if_name,ip6_autocfg_enabled,linux_name,mac_address,port_uuid,vnic_networks,vrf_ref
0,Unknown,True,False,eth1,False,eth4,00:0c:29:a7:83:8a,Unknown,"[{'ip': {'ip_addr': {'addr': '192.168.1.42', '...",https://192.168.1.15/api/vrfcontext/vrfcontext...


In [176]:
se1_vnic_to_migrate_config = se1_data_vnic_to_migrate[0]["vnic_networks"][0]["ip"]
se1_vnic_to_migrate_config_df = pd.DataFrame(se1_vnic_to_migrate_config)
# Display results in tab format
se1_vnic_to_migrate_config_df

,ip_addr,mask
addr,192.168.1.42,24
type,V4,24


In [177]:
# Select Interface to be migrated (ensure you are picking if_name and not linux_name column)
if_name_to_migrate = "eth1"


se2_data_vnic_to_migrate = [ d for d in se2_data_vnics if d.get("if_name") == if_name_to_migrate]
se2_data_vnic_to_migrate_df = pd.DataFrame(se2_data_vnic_to_migrate)

# Display results in tab format
se2_data_vnic_to_migrate_df

,adapter,connected,dhcp_enabled,if_name,ip6_autocfg_enabled,linux_name,mac_address,port_uuid,vnic_networks,vrf_ref
0,Unknown,True,False,eth1,False,eth4,00:0c:29:a7:61:3e,Unknown,"[{'ip': {'ip_addr': {'addr': '192.168.1.43', '...",https://192.168.1.15/api/vrfcontext/vrfcontext...


In [179]:
se2_vnic_to_migrate_config = se2_data_vnic_to_migrate[0]["vnic_networks"][0]["ip"]
se2_vnic_to_migrate_config_df = pd.DataFrame(se2_vnic_to_migrate_config)
# Display results in tab format
se2_vnic_to_migrate_config_df

,ip_addr,mask
addr,192.168.1.43,24
type,V4,24


In [180]:
print ("The following interfaces has been selected for migration")
print()
print("For the first Service Engine with name \033[1m"+se_data[0]["name"]+"\033[0m")
print("The interface \033[1m" + if_name_to_migrate + "\033[0m has been selected with following configuration")
print("   IP Address \033[1m" + se1_vnic_to_migrate_config["ip_addr"]["addr"] +"\033[0m")
print("   Mask Length \033[1m" + str(se1_vnic_to_migrate_config["mask"]) +"\033[0m")
print("   MAC-ADDRESS \033[1m" + se1_data_vnic_to_migrate[0]["mac_address"] +"\033[0m")
print()
print("For the second Service Engine with name \033[1m"+se_data[1]["name"]+"\033[0m")
print("The interface \033[1m" + if_name_to_migrate + "\033[0m has been selected with following configuration")
print("   IP Address \033[1m" + se2_vnic_to_migrate_config["ip_addr"]["addr"] +"\033[0m")
print("   Mask Length \033[1m" + str(se2_vnic_to_migrate_config["mask"]) +"\033[0m")
print("   MAC-ADDRESS \033[1m" + se2_data_vnic_to_migrate[0]["mac_address"] +"\033[0m")

The following interfaces has been selected for migration

For the first Service Engine with name fd38:9bda:673d:0:20c:29ff:fea7:8358
The interface eth1 has been selected with following configuration
   IP Address 192.168.1.42
   Mask Length 24
   MAC-ADDRESS 00:0c:29:a7:83:8a

For the second Service Engine with name fd38:9bda:673d:0:20c:29ff:fea7:610c
The interface eth1 has been selected with following configuration
   IP Address 192.168.1.43
   Mask Length 24
   MAC-ADDRESS 00:0c:29:a7:61:3e


### TARGET SERVICE ENGINE GROUP

In [181]:
# Set target name
target_segroup = "MAD-SEG001"

In [183]:
# GET Service Engine Group Configuration

# Define GET request parameters
url_path = "serviceenginegroup"
query = {
   "skip_default": "true",
   "name": target_segroup
}
# Send GET Request
resp = api.get(url_path, params=query)
print ("Request sent to URL: " + resp.url)

# Control Response Status Code
if resp.status_code in range(200, 299):
   print(resp)
   print(resp.reason)
    
   # Convert response JSON into Python Dictionary
   resp_data = json.loads(resp.text)

   # Extract Result
   resp_data = resp_data["results"]
   resp_names = [resp_name["name"] for resp_name in resp_data]
   print()
   print("The following " + url_path + " names has been found:")
   print(resp_names)
else:
    print('Error in GET request '+url_path+' :%s' % resp.text)

# Save Result
target_segroup_data = resp_data[0]

Request sent to URL: https://192.168.1.15/api/serviceenginegroup?skip_default=true&name=MAD-SEG001
<Response [200]>
OK

The following serviceenginegroup names has been found:
['MAD-SEG001']


In [186]:
# Extract related Service Engine

# GET Service Engine Related Configuration

segroup_data = target_segroup_data
# Define GET request parameters
se_group_uuid=segroup_data["uuid"]
url_path = "serviceengine"
query = {
   "skip_default": "true",
   "refers_to": f"serviceenginegroup:{se_group_uuid}"
}

# Send GET Request
resp = api.get(url_path, params=query)
print ("Request sent to URL: " + resp.url)

# Control Response Status Code
if resp.status_code in range(200, 299):
   print(resp)
   print(resp.reason)
    
   # Convert response JSON into Python Dictionary
   resp_data = json.loads(resp.text)

   # Extract Result
   resp_data = resp_data["results"]
   resp_names = [resp_name["name"] for resp_name in resp_data]
   print()
   print("The following " + url_path + " names has been found:")
   print(resp_names)
else:
    print('Error in GET request '+url_path+' :%s' % resp.text)

# Save Result
target_se_data = resp_data

Request sent to URL: https://192.168.1.15/api/serviceengine?skip_default=true&refers_to=serviceenginegroup%3Aserviceenginegroup-e5b13687-efbb-4417-9302-b1d37c367c8b
<Response [200]>
OK

The following serviceengine names has been found:
['fd38:9bda:673d:0:20c:29ff:fec4:a7af', 'fd38:9bda:673d:0:20c:29ff:feba:47eb']


In [187]:
# Extract Interface Information from first SE
se_data = target_se_data

se1_data_vnics = se_data[0]["data_vnics"]
se1_df = pd.DataFrame(se1_data_vnics)
se1_df

,adapter,connected,dhcp_enabled,if_name,ip6_autocfg_enabled,linux_name,mac_address,port_uuid,vrf_ref,vnic_networks
0,Unknown,True,False,eth9,False,eth5,00:0c:29:c4:a7:09,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
1,Unknown,True,False,eth3,False,eth3,00:0c:29:c4:a7:b9,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
2,Unknown,True,False,eth4,False,eth6,00:0c:29:c4:a7:c3,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
3,Unknown,True,False,eth6,False,eth8,00:0c:29:c4:a7:cd,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
4,Unknown,True,False,eth8,False,eth1,00:0c:29:c4:a7:d7,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
5,Unknown,True,NaN,eth1,False,eth4,00:0c:29:c4:a7:e1,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,"[{'ip': {'ip_addr': {'addr': '192.168.1.109', ..."
6,Unknown,True,False,eth2,False,eth7,00:0c:29:c4:a7:eb,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
7,Unknown,True,False,eth5,False,eth9,00:0c:29:c4:a7:f5,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
8,Unknown,True,False,eth7,False,eth2,00:0c:29:c4:a7:ff,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN


In [189]:
# Extract Interface Information from second SE
se2_data_vnics = se_data[1]["data_vnics"]
se2_df = pd.DataFrame(se2_data_vnics)
se2_df

,adapter,connected,dhcp_enabled,if_name,ip6_autocfg_enabled,linux_name,mac_address,port_uuid,vrf_ref,vnic_networks
0,Unknown,True,False,eth6,False,eth8,00:0c:29:ba:47:09,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
1,Unknown,True,False,eth8,False,eth1,00:0c:29:ba:47:13,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
2,Unknown,True,False,eth1,False,eth4,00:0c:29:ba:47:1d,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,"[{'ip': {'ip_addr': {'addr': '192.168.1.41', '..."
3,Unknown,True,False,eth2,False,eth7,00:0c:29:ba:47:27,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
4,Unknown,True,False,eth5,False,eth9,00:0c:29:ba:47:31,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
5,Unknown,True,False,eth7,False,eth2,00:0c:29:ba:47:3b,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
6,Unknown,True,False,eth9,False,eth5,00:0c:29:ba:47:45,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
7,Unknown,True,False,eth3,False,eth3,00:0c:29:ba:47:f5,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
8,Unknown,True,False,eth4,False,eth6,00:0c:29:ba:47:ff,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN


In [194]:
# Select one FREE interface that will be used as target
if_name_target = "eth2"

In [ ]:
# Assuming IP Address will be REUSED!!!
#  1.- Migrate VS (change SEGroup)
#  2.- Migrate SEs interfaces
#  3.- Migrate Network Services (if exists)
# Disable MIGRATING SE INTERFACES TO AVOID ITF CLASHING

## Migrate Service Engine Interfaces

In [195]:
# Display gathered information prior to execute next cell


print ("The following interfaces has been selected for migration")
print()
print("For the first Service Engine with name \033[1m"+se_data[0]["name"]+"\033[0m")
print("The interface \033[1m" + if_name_to_migrate + "\033[0m has been selected with following configuration")
print("   IP Address \033[1m" + se1_vnic_to_migrate_config["ip_addr"]["addr"] +"\033[0m")
print("   Mask Length \033[1m" + str(se1_vnic_to_migrate_config["mask"]) +"\033[0m")
print("   MAC-ADDRESS \033[1m" + se1_data_vnic_to_migrate[0]["mac_address"] +"\033[0m")
print()
print("For the second Service Engine with name \033[1m"+se_data[1]["name"]+"\033[0m")
print("The interface \033[1m" + if_name_to_migrate + "\033[0m has been selected with following configuration")
print("   IP Address \033[1m" + se2_vnic_to_migrate_config["ip_addr"]["addr"] +"\033[0m")
print("   Mask Length \033[1m" + str(se2_vnic_to_migrate_config["mask"]) +"\033[0m")
print("   MAC-ADDRESS \033[1m" + se2_data_vnic_to_migrate[0]["mac_address"] +"\033[0m")
print()
print("---------------------")
print("Above information will be migrated to a new Service Engine Group \033[1m" + target_segroup+"\033[0m")
print("The interface \033[1m" + if_name_target + "\033[0m will be configured with above information.")
print("---------------------")
print("WARNING!!! Before proceeding migrated interfaces in the migrating SE's will be disabled")

The following interfaces has been selected for migration

For the first Service Engine with name fd38:9bda:673d:0:20c:29ff:fec4:a7af
The interface eth1 has been selected with following configuration
   IP Address 192.168.1.42
   Mask Length 24
   MAC-ADDRESS 00:0c:29:a7:83:8a

For the second Service Engine with name fd38:9bda:673d:0:20c:29ff:feba:47eb
The interface eth1 has been selected with following configuration
   IP Address 192.168.1.43
   Mask Length 24
   MAC-ADDRESS 00:0c:29:a7:61:3e

---------------------
Above information will be migrated to a new Service Engine Group MAD-SEG001
The interface eth2 will be configured with above information.
WARNING!!! Before proceeding migrated interfaces in the migrating SE's will be disabled


In [271]:
# Select one FREE interface that will be used as target
if_name_target = "eth2"

for d in migrating_se_data:
    se_uuid = d.get("uuid")
    print(se_uuid)

se-564d63c6-3882-ca52-7be5-b04c9ba78358
se-564d0f61-3a27-ff26-8af7-306296a7610c


In [281]:
# Create dictionary list containint pair of migrating-target SEs  
se_pairs =[
     {"migrating_se_uuid": migrating_se_data[0]["uuid"],
      "target_se_uuid": target_se_data[0]["uuid"]
     },
     {"migrating_se_uuid": migrating_se_data[1]["uuid"],
      "target_se_uuid": target_se_data[1]["uuid"]
     }
]

[{'migrating_se_uuid': 'se-564d63c6-3882-ca52-7be5-b04c9ba78358',
  'target_se_uuid': 'se-564dcb90-ff93-dc99-bf4a-62907bc4a7af'},
 {'migrating_se_uuid': 'se-564d0f61-3a27-ff26-8af7-306296a7610c',
  'target_se_uuid': 'se-564d5830-e7a9-edc0-194c-c76236ba47eb'}]

In [284]:
for i, d in enumerate()

In [272]:
key_to_check = "if_name"
value_to_match = if_name_target
key_to_modify = "enabled" # If it does not exists key will be created
new_value = False

                  
json_data = [ d for d in migrating_se_data if d.get("uuid") == uuid1]
json_data_vnics = json_data[0]["data_vnics"]
json_data_vnics[5]

{'adapter': 'Unknown',
 'connected': True,
 'dhcp_enabled': False,
 'if_name': 'eth2',
 'ip6_autocfg_enabled': False,
 'linux_name': 'eth7',
 'mac_address': '00:0c:29:a7:61:48',
 'port_uuid': 'Unknown',
 'vrf_ref': 'https://192.168.1.15/api/vrfcontext/vrfcontext-e69db980-a4b4-4097-8717-7702fdba30b6',
 'enabled': True}

In [273]:

# Loop through each dictionary in the list
for d in json_data_vnics:
    # Check if the key_to_check matches the value_to_match
    if d.get(key_to_check) == value_to_match:
        # Update the value of key_to_modify
        d[key_to_modify] = new_value

json_data_vnics[5]


{'adapter': 'Unknown',
 'connected': True,
 'dhcp_enabled': False,
 'if_name': 'eth2',
 'ip6_autocfg_enabled': False,
 'linux_name': 'eth7',
 'mac_address': '00:0c:29:a7:61:48',
 'port_uuid': 'Unknown',
 'vrf_ref': 'https://192.168.1.15/api/vrfcontext/vrfcontext-e69db980-a4b4-4097-8717-7702fdba30b6',
 'enabled': False}

In [274]:
json_data_updated_vnics = {"data_vnics": json_data_vnics}
json_data_updated_vnics

{'data_vnics': [{'adapter': 'Unknown',
   'connected': True,
   'dhcp_enabled': False,
   'if_name': 'eth3',
   'ip6_autocfg_enabled': False,
   'linux_name': 'eth3',
   'mac_address': '00:0c:29:a7:61:16',
   'port_uuid': 'Unknown',
   'vrf_ref': 'https://192.168.1.15/api/vrfcontext/vrfcontext-e69db980-a4b4-4097-8717-7702fdba30b6'},
  {'adapter': 'Unknown',
   'connected': True,
   'dhcp_enabled': False,
   'if_name': 'eth4',
   'ip6_autocfg_enabled': False,
   'linux_name': 'eth6',
   'mac_address': '00:0c:29:a7:61:20',
   'port_uuid': 'Unknown',
   'vrf_ref': 'https://192.168.1.15/api/vrfcontext/vrfcontext-e69db980-a4b4-4097-8717-7702fdba30b6'},
  {'adapter': 'Unknown',
   'connected': True,
   'dhcp_enabled': False,
   'if_name': 'eth6',
   'ip6_autocfg_enabled': False,
   'linux_name': 'eth8',
   'mac_address': '00:0c:29:a7:61:2a',
   'port_uuid': 'Unknown',
   'vrf_ref': 'https://192.168.1.15/api/vrfcontext/vrfcontext-e69db980-a4b4-4097-8717-7702fdba30b6'},
  {'adapter': 'Unknown'

In [275]:
json_data

[{'_last_modified': '1723585446326584',
  'active_tags': ['ACTIVE_STANDBY_SE_1'],
  'cloud_ref': 'https://192.168.1.15/api/cloud/cloud-baf1f7f6-18ff-46cb-a134-6154d9af52a1',
  'data_vnics': [{'adapter': 'Unknown',
    'connected': True,
    'dhcp_enabled': False,
    'if_name': 'eth3',
    'ip6_autocfg_enabled': False,
    'linux_name': 'eth3',
    'mac_address': '00:0c:29:a7:61:16',
    'port_uuid': 'Unknown',
    'vrf_ref': 'https://192.168.1.15/api/vrfcontext/vrfcontext-e69db980-a4b4-4097-8717-7702fdba30b6'},
   {'adapter': 'Unknown',
    'connected': True,
    'dhcp_enabled': False,
    'if_name': 'eth4',
    'ip6_autocfg_enabled': False,
    'linux_name': 'eth6',
    'mac_address': '00:0c:29:a7:61:20',
    'port_uuid': 'Unknown',
    'vrf_ref': 'https://192.168.1.15/api/vrfcontext/vrfcontext-e69db980-a4b4-4097-8717-7702fdba30b6'},
   {'adapter': 'Unknown',
    'connected': True,
    'dhcp_enabled': False,
    'if_name': 'eth6',
    'ip6_autocfg_enabled': False,
    'linux_name': '

In [278]:
json_data_dict=json_data[0]
json_data_dict.update(json_data_updated_vnics)
json_data_dict
#["data_vnics"][5]

{'_last_modified': '1723585446326584',
 'active_tags': ['ACTIVE_STANDBY_SE_1'],
 'cloud_ref': 'https://192.168.1.15/api/cloud/cloud-baf1f7f6-18ff-46cb-a134-6154d9af52a1',
 'data_vnics': [{'adapter': 'Unknown',
   'connected': True,
   'dhcp_enabled': False,
   'if_name': 'eth3',
   'ip6_autocfg_enabled': False,
   'linux_name': 'eth3',
   'mac_address': '00:0c:29:a7:61:16',
   'port_uuid': 'Unknown',
   'vrf_ref': 'https://192.168.1.15/api/vrfcontext/vrfcontext-e69db980-a4b4-4097-8717-7702fdba30b6'},
  {'adapter': 'Unknown',
   'connected': True,
   'dhcp_enabled': False,
   'if_name': 'eth4',
   'ip6_autocfg_enabled': False,
   'linux_name': 'eth6',
   'mac_address': '00:0c:29:a7:61:20',
   'port_uuid': 'Unknown',
   'vrf_ref': 'https://192.168.1.15/api/vrfcontext/vrfcontext-e69db980-a4b4-4097-8717-7702fdba30b6'},
  {'adapter': 'Unknown',
   'connected': True,
   'dhcp_enabled': False,
   'if_name': 'eth6',
   'ip6_autocfg_enabled': False,
   'linux_name': 'eth8',
   'mac_address': '0

In [196]:
print ("Disabling interface at SE1. This would affect existing Virtual Services")

#Send POST information via POST

url_path = "serviceengine"

resp = api.post (url_path, data=json.dumps(data))

if resp.status_code in range(200, 299):
    print(resp)
    print('- New '+url_path+' named '+data['name'], resp.reason)#, resp.text)
else:
    print('Error in creating '+url_path+' :%s' % resp.text)


Disabling interface at SE1. This would affect existing Virtual Services


[{'_last_modified': '1723585420269617',
  'cloud_ref': 'https://192.168.1.15/api/cloud/cloud-baf1f7f6-18ff-46cb-a134-6154d9af52a1',
  'data_vnics': [{'adapter': 'Unknown',
    'connected': True,
    'dhcp_enabled': False,
    'if_name': 'eth9',
    'ip6_autocfg_enabled': False,
    'linux_name': 'eth5',
    'mac_address': '00:0c:29:c4:a7:09',
    'port_uuid': 'Unknown',
    'vrf_ref': 'https://192.168.1.15/api/vrfcontext/vrfcontext-e69db980-a4b4-4097-8717-7702fdba30b6'},
   {'adapter': 'Unknown',
    'connected': True,
    'dhcp_enabled': False,
    'if_name': 'eth3',
    'ip6_autocfg_enabled': False,
    'linux_name': 'eth3',
    'mac_address': '00:0c:29:c4:a7:b9',
    'port_uuid': 'Unknown',
    'vrf_ref': 'https://192.168.1.15/api/vrfcontext/vrfcontext-e69db980-a4b4-4097-8717-7702fdba30b6'},
   {'adapter': 'Unknown',
    'connected': True,
    'dhcp_enabled': False,
    'if_name': 'eth4',
    'ip6_autocfg_enabled': False,
    'linux_name': 'eth6',
    'mac_address': '00:0c:29:c4:a7: